# Notebook for the development of module "scheduling_problem" of AI for carbon reduction project

WAYS TO SOLVE THE PROBLEMS:
- glpk (1 and 3) =  a software package designed to solve large-scale linear programming (LP), mixed integer programming (MIP), and related problems
- 3 local search = solve initially with glpk (or with something that you know is right), and then try to do small changes on number of panels and batteries (almost all decreasing, something increasing)
- SCIP (4_1 lin prog) = a method used to create a solver instance for solving mixed integer programming (MIP) problems using the SCIP solver. Essentially same as glpk
- CpSolver (4_1 sat) = un po' come gli altri, ma in teoria è più veloce
- enhanced (always uses CpSolver)
   - improved_backtracking = most constrained variable, backjumping, no good learning, least constraining value, node consistency, constraint propagation
   - constraint propagation = node consistency, constraint propagation 
   - simulated annealing = start from a random solution, see neighbors, choose a random one and change solution only if it is a better one (this also depends on the temperature)
   - local beam = with k solutions at random pick all their neighbors and keep the best k among them, and continue
   - genetic algorithm = given a population of solutions, keep the best 20% and create new solutions
   - min conflicts = starting from random, get one of the variables that has conflicts and make it have a value to reduce the conflicts
   - find_min_original = 4_1 sat
   - stochastic hill climbing with random restart = accepts a neighbor if it's better, of if it's worse than only with a certain probability
   - tabu search = it has restarts, occasionally it makes some random moves, gets the neighbor with the least conflicts, 


# General model
We list below the common sets, parameters, variables and constraints that will be used in all the models.

## Sets
$$
T = \{0, ..., T_H-1\} = \text{Discrete time steps in the scheduling horizon (e.g., minutes).}
$$
$$
I = \{1, ...., N_M\} = \text{Set of machines.}
$$
$$
J_i = \{1, ..., N_i\} = \text{Set of jobs to be processed by machine $i$.}
$$
## Parameters
For each machine:
$$
 e_{i} = \text{energy used by the machine $i$ per unit of time}
 $$
 $$
 f_{i} = \text{fixed cost for starting the machine $i$ at any time}
 $$
 $$
d_i = \text{duration of any of the job of machine $i$}
$$
$$
N_i = \text{number of jobs to complete for machine $i$}
$$
$$
c_i = \text{time of cooldown for machine $i$}
$$
$$
T_{j,i} = \text{threshold for job $j$ of machine $i$}
$$
For each time step:
$$
p_t = \text{energy produced at time $t$ by one solar panel}
$$
$$
m_t = \text{maximum energy that we can use at time $t$}
$$
For each solar panel:
$$
c_p = \text{cost of a solar panel}
$$
For each battery:
$$
c_b = \text{cost of a battery}
$$
$$
B = \text{capacity of a battery}
$$

## Variables
$$
x_{itj} = \text{if machine $i$ is working at time $t$ for job $j$}
$$
$$
y_{itj} = \text{if machine $i$ starts working at time $t$ for job $j$}
$$
$$
z_{t} = \text{value of objective function at time t, real number}
$$

## Constraints

1. Every machine $i \in I$ must do the required number $N_i$ of jobs with a certain duration $d_i$. For every $i \in I$ and $j \in J_i$:
$$
\sum_t x_{itj} \ge  d_i
$$
2. Knapsack contraint that says that not all machines can run at the same time due to max power load for slot (parameter $m_t$, which could depend on time). For every $t \in T$:
$$
   \sum_i \sum_j e_{i}x_{itj} \le m_t
$$
3. Some machines may be allowed to run only at specific times for noise or maintenance windows / downtime or worker availability. For some $t \in T$ and $i \in I$:
   $$
   x_{itj} = 0 \, \, \forall j \in J_i
   $$
4. Shared resources: some machines can't run in the same time slot $t \in T$ because of shared resources. Given a set $M = \{M'$ : if $k,l \in M'$ then $k$ and $l$ have shared resources$\}$ and $\forall t \in T$:
   $$
   \sum_{i \in M'} \sum_j x_{itj} \le 1  \,\, \forall M' \in M\\
   $$
5. A machine works only if it has just started or it was already working. For every $i \in I$, for every $j \in J$ and for every $t \in T$:
$$            
y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j}
$$
6. Machine dependencies: one machine starts only after another has finished. Given a set $M = \{(k,l)$ : $l$ must start its job only after $k$ has finished its$\}$ and for every time $t \in T$.
$$
y_{ltj} \cdot d_k \le  \sum_{t' < t} (x_{kt'j}) \text{ for every dependency }(k,l) \in M \text{ and for every $j \in J_l$}
$$
7. Cooldown Periods: some machines require a cooldown phase between uses (if this is not needed, set $c_i = 0$). For every $i \in I$ and for every $t \in T$:
$$
   c_i \cdot \sum_j y_{itj} \le \sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})
$$
8. Job deadlines: all jobs must finish before a specific time. For every $i \in I$ and for every $j \in J$, calling $T_{j,i}$ the threshold for job $j$ on machine $i$:
$$
     \sum_{t' \le T_{j,i}} x_{it'j} \ge d_i
$$
9. Only one job at a time. For every $i \in I$ and for every $t \in T$:
$$
    \sum_j x_{itj} \le 1
$$
$$
    \sum_j y_{itj} \le 1
$$


## PHASE 1
Determine the minimum number of solar panels ($M$) and battery units ($N$) for 100% self-sufficiency.

### Additional variables
$$
N = \text{how many batteries to buy}
$$
$$
M = \text{how many panels to buy}
$$
$$
s_t = \text{how much energy we accumulated till time $t$ in the batteries}
$$
### Objective function
$$
\min Nc_b+Mc_p
$$
### Additional constraints
1. No imported energy. For every $t \in T$:
$$
\sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le  0
$$
2. Constraints on the level of energy accumulated in the batteries. For every $t \in T$:
$$
         s_t \le  \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})
$$
$$
         0 \le s_t \le  NB
$$

### Final model
The following is the final model that is solved in phase 1.

$$
\left\{
  \begin{array}{rcr}
        \min Nc_b+Mc_p \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & 0  & \text{for every t}\\
         s_t \le & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})  & \text{for every t. Should be an equal but in that case it could be more than NB}\\
         0 \le s_t \le & NB & \text{at every time t}\\
         \sum_t x_{itj} \ge & d_i & \text{for every i and j}\\
         \sum_j x_{itj} \le & 1 & \text{for every i and t}\\
         \sum_j y_{itj} \le & 1 & \text{for every i and t}\\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t}\\
         \sum_{t' \le T} x_{it'j} \ge & d_i & \text{ at some time T (threshold) and for every j and i}\\         
  \end{array}
\right.
$$

## PHASE 2
Taking the number of panels $M^*$ and the number of batteries $N^*$ that result from phase 1, compute the number of years that our company could work continuing to pay external energy instead of buying solar panels.

Let $c_{annual}$ be the annual cost of the imported energy.

$$
horizon = H = \frac{c_bN^*+c_pM^*}{c_{annual}}
$$

## PHASE 3
Within the budget implied by Phase 2’s horizon, find optimal $N$ and $M$ to minimize total energy cost (capital + operational).

$$
H = \text{massimo orizzonte temporale}
$$

$$
\left\{
  \begin{array}{rcr}
    \min Nc_b + Mc_p + \sum_{0\le t\le H} z_t \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & z_t & \text{at every time t} \\
         V_t = & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \text{for every t} \\
         V_t \ge & -BIG_M (1 - b_t) & \text{for every t, b\_t = 1 if V\_t > 0} \\
         V_t \le & BIG_M b_t & \text{for every t, b\_t = 0 if V\_t < 0} \\
         s_t \le & V_t + BIG_M (1 - b_t) & \text{for every t} \\
         s_t \le & BIG_M b_t & \text{for every t} \\
         0 \le s_t \le & NB & \text{at every time t}\\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i} \\
         \sum_j x_{itj} \le & 1 & \text{for every i and t} \\
         \sum_j y_{itj} \le & 1 & \text{for every i and t} \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t} \\
         \sum_{t' \le T} x_{itj} = & d_i & \text{ at some time T (threshold) and for every j and i} \\         
  \end{array}
\right.
$$

## PHASE 4
Now that we fixed N and M, we need to obtain an optimal scheduling that reduces the usage of external energy.

This phase can be divided in 2.

### 4.1
If I need to not use at all any form of external energy.

$$
\left\{
  \begin{array}{rcr}
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & 0  & \text{for every t}\\
         s_t \le & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j})  & \text{for every t. Should be an equal but in that case it could be more than NB}\\
         0 \le s_t \le & NB & \text{at every time t}\\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i}\\
         \sum_j x_{itj} \le & 1 & \text{for every i and t}\\
         \sum_j y_{itj} \le & 1 & \text{for every i and t}\\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t}\\
         \sum_{t' \le T} x_{it'j} = & d_i & \text{ at some time T (threshold) and for every j and i}\\         
  \end{array}
\right.
$$

### 4.3
Minimize the external energy used.

$$
\left\{
  \begin{array}{rcr}
    \min \sum_t z_t \\
         \sum_i\sum_j (e_{i}x_{itj} +f_i y_{itj} - Mp_t - s_t) \le & z_t & \text{at every time t} \\
         V_t = & \sum_{t'<t}\sum_i\sum_j (Mp_{t'} - e_{i}x_{it'j} - f_i y_{it'j}) & \text{for every t} \\
         V_t \ge & -BIG_M (1 - b_t) & \text{for every t, b\_t = 1 if V\_t > 0} \\
         V_t \le & BIG_M b_t & \text{for every t, b\_t = 0 if V\_t < 0} \\
         s_t \le & V_t + BIG_M (1 - b_t) & \text{for every t} \\
         s_t \le & BIG_M b_t & \text{for every t} \\
         0 \le s_t \le & NB & \text{at every time t}\\
         0 \le & z_t & \text{at every time t} \\
         \sum_t \sum_j x_{itj} \ge & n_i d_i & \text{for every i} \\
         \sum_j x_{itj} \le & 1 & \text{for every i and t} \\
         \sum_j y_{itj} \le & 1 & \text{for every i and t} \\
         \sum_i \sum_j e_{i}x_{itj} \le & m_t & \text{for every t. Max energy} \\
         \sum_j x_{itj} = & 0 & \text{ for some machine i and time t} \\
         \sum_{i \in M'} \sum_j x_{itj} \le & 1 & \text{ for some subsets M', M'',.. with shared resources, at every time t}\\
         y_{itj} \le x_{itj} \le y_{itj}+x_{i(t-1)j} \\            
         y_{(k+1)tj} \le  & \frac{\sum_{t' < t} (x_{kt'j})}{d_k} & \text{ for some dependency }(k,k+1) \in M \text{ between machines and for every j and t}\\
         \sum_j y_{itj} \le & \frac{\sum_{t-c_i \le t' < t}\sum_j (1-x_{it'j})}{c_i} & \text{cooldown for some i and for every t} \\
         \sum_{t' \le T} x_{itj} = & d_i & \text{ at some time T (threshold) and for every j and i} \\         
  \end{array}
\right.
$$


### 3. Algorithms to solve the problem
See what we did in class

LOCAL SEARCH:
* we can solve a CSP and then use a local search to optimize. IDEA OF LOCAL SEARCH: start from a feasible point, and then see what are its neighbors. Find the highest value neighbor and see if its value is higher than the one we are using right now. If yes, go to it. (HILL-CLIMBING, problem is I can get stuck in a local minimum/maximum). So we can use:
   * stochastic hill-climbing: choose a neighbor at random from those that are better
   * do a series of hill-climbing and return the best
   * simulated annealing: do a move at random. Accepted if improving, if not then it is accepted with a probability that depends on the time. (REALLY OFTEN USED IN AIRLINE SCHEDULING).
   * LOCAL BEAM: start from K states, at each step go in all neighbors till I find the goal
   * GENETIC ALGORITHMS: combine various states to obtain only one

CSP:
 * to get one value per variable that satisfies all the constraints. The idea is that we use general purpose heuristics rather than problem-specific ones. \
A state is an assignment of values to some or all variables. We want consistent (doesn't violate any constraint) and complete states. \
ANY CSP CAN BE CONVERTED TO A CSP WITH ONLY BINARY CONSTRAINTS.
   * BACKTRACKING SEARCH: start from empty assignment and assign a value that do not conflict (depth first). Backtrack if no legal values.
   * IMPROVED BACKTRACKING:
     * always choose the variable with fewest legal values (break ties choosing the variable involved in the most constraints with unassigned variables).
     * Then, chosen a variable, choose the least constraining value (the one that rules out the fewest values in the other variables).
     * instead of chronological backtracking (backtrack to the previous variable), do BACKJUMPING: backtrack to a variable that might fix the problem.
     * add the constraint on the variables that caused the problem (NO-GOOD) to avoid to redo the error
     * FORWARD CHECKING: if any of the remaining variable has no legal values, terminate.
     * CONSTRAINT PROPAGATION: enforce local consistency. It's a preprocessing step (or also done after assigning a new value). This doesn't guarantee to solve the problem, but helps pruning it:
       * node-consistency: a variable is node-consistent if all the values in the domain satisfy the variable's unary constraints. If I check this, I can delete all the unary constraints
       * arc-consistency (AC-3): a variable is arc-consistent if all the values in the domain satisfy the binary constraints (we are assuming that every other constraint is converted to binary).
       * path-consistency: consistency between more than 2 variables
   * LOCAL SEARCH: start from something that violates the constraints and try to do small changes (one value to one variable) to get the right solution. The search space is all the complete assignments, we need to find one that is consistent. \
To evaluate states we associate the number of constraints it violates.
     * MIN-CONFLICTS: We choose the next value to minimize the number of conflicts.
     * IMPROVED:
       * RESTART: re-initialize the search after tot steps
       * RANDOM WALK: choose something at random but based on probability
       * TABU SEARCH: remember a list of forbidden states (few) to avoid cycling.
       * CONSTRAINT WEIGHTING: eah constraint has a weight. At each step choose a variable with lowest total weight of all violated constraints. (low weight = more important).

LOCAL SEARCH is really useful if I found a good scheduling but there are some minor changes during the week.

CONNECTED COMPONENTS: of the constraint graph are different problems that can be solved alone. For example if it's a tree it's easier. SO WE CAN FIX SOME VARIABLES TO OBTAIN A TREE (TREE DECOMPOSITION).